In [2]:
# Import the CARLA Python API library and some utils
import carla 
import math 
import random 
import time 

In [3]:
client = carla.Client('localhost', 2000)
world = client.get_world()

In [4]:
# Get the blueprint library and the spawn points for the map
bp_lib = world.get_blueprint_library() 
spawn_points = world.get_map().get_spawn_points() 

In [5]:
# Get the blueprint for the vehicle you want
vehicle_bp = bp_lib.find('vehicle.lincoln.mkz_2020') 

# Try spawning the vehicle at a randomly chosen spawn point
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

In [7]:
# Move the spectator behind the vehicle 
spectator = world.get_spectator() 
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4,z=2.5)),vehicle.get_transform().rotation) 
spectator.set_transform(transform) 

In [8]:
# Add traffic to the simulation
for i in range(30): 
    vehicle_bp = random.choice(bp_lib.filter('vehicle')) 
    npc = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points)) 

In [9]:
# Set the all vehicles in motion using the Traffic Manager
for v in world.get_actors().filter('*vehicle*'): 
    v.set_autopilot(True) 

In [10]:
blueprint = world.get_blueprint_library().find('sensor.other.imu')

imu_location = carla.Location(x=0.0, y=0.0, z=1.0)
imu_rotation = carla.Rotation(pitch=0.0, roll=0.0, yaw=0.0)
imu_transform = carla.Transform(imu_location, imu_rotation)
imu_sensor = world.spawn_actor(blueprint, imu_transform, attach_to=vehicle)

In [11]:
def imu_callback(imu_data):
    linear_acceleration = imu_data.accelerometer
    angular_velocity = imu_data.gyroscope
    orientation = imu_data.compass

    # Write IMU data to file
    with open('imu_data.txt', 'a') as file:
        file.write(f'time: {imu_data.timestamp}, x-acceleration: {imu_data.accelerometer.x}, y-acceleration : {imu_data.accelerometer.y}, z-acceleration : {imu_data.accelerometer.z}\n')
        
imu_sensor.listen(imu_callback)

In [12]:
imu_sensor.destroy()

True

In [13]:
client.reload_world()

: 